In [ ]:
import pandas as pd
from pathlib import Path
from urllib.parse import quote_plus
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os

DIR_WORKSPACE = Path.cwd().parents[0]
DIR_DATA = DIR_WORKSPACE / "data"
DIR_REPORTS_CSV = DIR_DATA / "mse-daily-data"
DIR_OUTPUT = DIR_DATA / "output_combined_data"
DIR_COUNTERS = DIR_DATA / "output_combined_data"

In [ ]:
# Collect and combine all CSV files
all_csv_files = DIR_REPORTS_CSV.glob("*.csv")  
df_list = [pd.read_csv(f) for f in all_csv_files]

if df_list:  
    combined_df = pd.concat(df_list, ignore_index=True)
    print("Combined shape:", combined_df.shape)

    output_file = DIR_OUTPUT / "combined_reports.csv"
    combined_df.to_csv(output_file, index=False)
    print("Combined CSV saved at:", output_file)
else:
    print("No CSV files found in", DIR_REPORTS_CSV)


In [ ]:
# Load the combined CSV file into a DataFrame
combined_stock_df = pd.read_csv(output_file)

company_id_map = {
    "AIRTEL": "MWAIRT001156",
    "BHL": "MWBHL0010029",
    "FDHB": "MWFDHB001166",
    "FMBCH": "MWFMB0010138",
    "ICON": "MWICON001146",
    "ILLOVO": "MWILLV010032",
    "MPICO": "MWMPI0010116",
    "NBM": "MWNBM0010074",
    "NBS": "MWNBS0010105",
    "NICO": "MWNICO010014",
    "NITL": "MWNITL010091",
    "OMU": "ZAE000255360",
    "PCL": "MWPCL0010053",
    "STANDARD": "MWSTD0010041",
    "SUNBIRD": "MWSTL0010085",
    "TNM": "MWTNM0010126"
}

# Add a new column with the full name
combined_stock_df["counter_id"] = combined_stock_df["counter"].map(company_id_map)

In [ ]:
counters_df = pd.read_csv(DIR_COUNTERS / "counters_df.csv")

price_daily_columns = {
    "counter_id": "counter_id",
    "trade_date": "trade_date",
    "previous_closing_price": "open_mwk",
    "daily_range_high": "high_mwk",
    "daily_range_low": "low_mwk",
    "today_closing_price": "close_mwk",
    "volume_traded": "volume"
}

prices_daily_df = combined_stock_df.rename(columns=price_daily_columns)[list(price_daily_columns.values())]
prices_daily_df.to_csv(DIR_OUTPUT / "price_daily_data.csv", index=False)


In [ ]:
unique_tickers = counters_df['ticker'].unique()
print(unique_tickers)
print(unique_tickers.size)
unique_counters = combined_stock_df['counter'].unique()
print(unique_counters)
print(unique_counters.size)

In [ ]:
load_dotenv()  

PGHOST = os.getenv("PGHOST").strip()
PGPORT = os.getenv("PGPORT").strip()
PGPORT = int(''.join(filter(str.isdigit, PGPORT)))
PGDATABASE = os.getenv("PGDATABASE").strip()
PGUSER = os.getenv("PGUSER").strip()
PGPASSWORD = os.getenv("PGPASSWORD").strip()

print("PostgreSQL Connection Settings:")
print(f"Host: {PGHOST}")
print(f"Port: {PGPORT}")
print(f"Database: {PGDATABASE}")
print(f"User: {PGUSER}")
print(f"Password: {'[SET]' if PGPASSWORD else '[NOT SET]'}")

encoded_password = quote_plus(PGPASSWORD)
connection_string = f"postgresql+psycopg2://{PGUSER}:{encoded_password}@{PGHOST}:{PGPORT}/{PGDATABASE}"

# Mask password when printing
safe_connection_string = f"postgresql+psycopg2://{PGUSER}:****@{PGHOST}:{PGPORT}/{PGDATABASE}"
print("Connection psql string:", safe_connection_string)

# Use the real one for engine creation
engine = create_engine(connection_string, pool_pre_ping=True)


In [ ]:
with engine.connect() as conn:
    # Get current user and database
    who = conn.execute(text("SELECT current_user, current_database();")).fetchone()
    print("Connected as:", who)
    
    # List all tables in the public schema
    result = conn.execute(text("""
        SELECT table_schema, table_name
        FROM information_schema.tables
        WHERE table_schema = 'public'
        ORDER BY table_name;
    """)).fetchall()
    
    print("Tables in database:")
    for row in result:
        print(f"{row.table_schema}.{row.table_name}")

In [ ]:

#=======================================
# EMPTY TABLES BEFORE INSERTING NEW DATA
#=======================================

tables_to_truncate = ["public.counters", "public.prices_daily"]

with engine.connect() as conn:
    # Optional: check connection
    who = conn.execute(text("SELECT current_user, current_database();")).fetchone()
    print("Connected as:", who)

    # Truncate tables safely
    for table in tables_to_truncate:
        conn.execute(text(f"TRUNCATE TABLE {table} RESTART IDENTITY CASCADE;"))
        print(f"Emptied table: {table}")
    
    # Commit changes if using transactional engine
    conn.commit()


In [ ]:
create_counters_table = """
CREATE TABLE IF NOT EXISTS counters (
    counter_id TEXT PRIMARY KEY,
    ticker TEXT,
    name TEXT,
    date_listed DATE,
    listing_price NUMERIC(10,2)
);
"""

create_prices_daily_table = """
CREATE TABLE IF NOT EXISTS prices_daily (
    counter_id TEXT REFERENCES counters(counter_id),
    trade_date DATE,
    open_mwk NUMERIC(10,2),
    high_mwk NUMERIC(10,2),
    low_mwk NUMERIC(10,2),
    close_mwk NUMERIC(10,2),
    volume BIGINT,
    PRIMARY KEY (counter_id, trade_date)
);
"""

with engine.connect() as conn:
    conn.execute(text(create_counters_table))
    conn.execute(text(create_prices_daily_table))
    conn.commit()
    print("Tables created successfully!")


In [ ]:
#=============================
# ADD NEW COLUMN 'Economic Sector'
#==============================
add_sector_column = """
DO $$
BEGIN
    IF NOT EXISTS (
        SELECT 1 FROM information_schema.columns
        WHERE table_name='counters' AND column_name='sector'
    ) THEN
        ALTER TABLE counters ADD COLUMN sector TEXT;
    END IF;
END $$;
"""

with engine.connect() as conn:
    conn.execute(text(add_sector_column))
    conn.commit()
    print("✅ Added column 'sector' to counters table (if missing).")

In [ ]:
# Insert counters
counters_df.to_sql('counters', engine, if_exists='append', index=False)
# Insert daily prices
prices_daily_df.to_sql('prices_daily', engine, if_exists='append', index=False)

print("Data from CSV inserted successfully!")


In [ ]:
# -----------------------------
# UPDATE SECTOR INFORMATION
# -----------------------------
COMPANIES_BY_SECTOR = {
    "Banking": ["FDHB", "FMBCH", "NBM", "NBS", "NICO", "STANDARD"],
    "Telecom": ["AIRTEL", "TNM"],
    "Hospitality": ["BHL", "SUNBIRD"],
    "Real Estate": ["ICON", "MPICO"],
    "Manufacturing": ["ILLOVO"],
    "Investment": ["NITL", "OMU", "PCL"]
}

with engine.connect() as conn:
    for sector, tickers in COMPANIES_BY_SECTOR.items():
        for ticker in tickers:
            conn.execute(
                text("UPDATE counters SET sector = :sector WHERE ticker = :ticker"),
                {"sector": sector, "ticker": ticker}
            )
    conn.commit()
    print("✅ Updated sector information for all counters.")

In [ ]:
# Read counters table
counters_sample = pd.read_sql("SELECT * FROM counters LIMIT 16;", engine)
print("Counters table sample:")
counters_sample.sort_values(by='date_listed', ignore_index=True, inplace=True)
counters_sample


In [ ]:

# Read daily prices table
prices_sample = pd.read_sql("SELECT * FROM prices_daily LIMIT 10;", engine)
print("\n\nPrices table sample:")
prices_sample
